In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
train_chicken_dir = "/content/drive/MyDrive/data/chicken-images/data/train"
train_duck_dir = "/content/drive/MyDrive/data/duck-images/data/train"
val_chicken_dir = "/content/drive/MyDrive/data/chicken-images/data/val"
val_duck_dir = "/content/drive/MyDrive/data/duck-images/data/val"
test_chicken_dir = "/content/drive/MyDrive/data/chicken-images/data/test"
test_duck_dir = "/content/drive/MyDrive/data/duck-images/data/test"

In [3]:
from torch.utils.data import Dataset
from PIL import Image
import os

class DuckChickenDataset(Dataset):
    def __init__(self, chicken_dir, duck_dir, transform=None):
        self.transform = transform
        self.samples = []

        for img_name in os.listdir(chicken_dir):
            self.samples.append((os.path.join(chicken_dir, img_name), 0))  # label 0: chicken

        for img_name in os.listdir(duck_dir):
            self.samples.append((os.path.join(duck_dir, img_name), 1))  # label 1: duck

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [4]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = DuckChickenDataset(train_chicken_dir, train_duck_dir, transform)
val_dataset = DuckChickenDataset(val_chicken_dir, val_duck_dir, transform)
test_dataset = DuckChickenDataset(test_chicken_dir, test_duck_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


In [5]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 93.2MB/s]


In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

def train(model, train_loader, val_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}: Loss={running_loss:.4f}, Train Acc={correct/total:.4f}, Val Acc={val_acc:.4f}")

train(model, train_loader, val_loader, epochs=5)


Epoch 1: Loss=12.0772, Train Acc=0.8105, Val Acc=0.8820
Epoch 2: Loss=6.7523, Train Acc=0.9253, Val Acc=0.9130
Epoch 3: Loss=4.7423, Train Acc=0.9509, Val Acc=0.9317
Epoch 4: Loss=4.4988, Train Acc=0.9509, Val Acc=0.9317
Epoch 5: Loss=3.7562, Train Acc=0.9654, Val Acc=0.8882


In [7]:
from sklearn.metrics import classification_report

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print("Classification Report:\n")
print(classification_report(all_labels, all_preds, target_names=["chicken", "duck"]))


Classification Report:

              precision    recall  f1-score   support

     chicken       1.00      0.82      0.90       172
        duck       0.91      1.00      0.95       310

    accuracy                           0.94       482
   macro avg       0.95      0.91      0.93       482
weighted avg       0.94      0.94      0.93       482

